# 畢業流向（1年, 3年兩類） 101-103

In [8]:
'''
def1: 開檔,欄位刪除,重命名,排序,存檔輸出  
def2: 單選字串編碼,反向計分編碼,存檔輸出
def3: 複選字串編碼,答案存入欄位,存檔輸出(是否獨立皆可)
'''
'''
def4: shape轉置(單欄與多欄),存檔輸出(pivot或unstack方法)  
'''         #全部包含次數檢查（不含其他項）

'\ndef4: shape轉置(單欄與多欄),存檔輸出  \n'

In [10]:
#查看所有檔案(PATH)
!pip install glob      
import glob        #可在prompt或linux

filename=glob.glob('./IRdata/graduate/*')  #100-103為入學年,有1y,3y
filename   #filename list

  Could not find a version that satisfies the requirement glob (from versions: )
No matching distribution found for glob
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


['./IRdata/graduate\\graduate_100-3y.xlsx',
 './IRdata/graduate\\graduate_101-1y.xlsx',
 './IRdata/graduate\\graduate_101-3y.xlsx',
 './IRdata/graduate\\graduate_102-1y.xlsx',
 './IRdata/graduate\\graduate_103-1y.xlsx']

In [11]:
#創item list
items = ['第'+'%s'%i+'題' for i in range(1,15)]    #第13,14題的問題 1y,3y不同, 第14題只有3y有
print(items)  #全部問題

#分為單選,複選,反向計分
items_s = [] 
items_1 = items[0:2]
items_2 = items[3:5]
items_3 = items[6:10]
items_s.extend(items_1)
items_s.extend(items_2)
items_s.extend(items_3)
items_s.append(items[11]) 
items_m = [items[2],items[5],items[10],items[12],items[13]] 
items_scoring = [items[6],items[8],items[9]]  

print(items_s)   #單選               #3y的第13題為單選,為反向計分,1y的第13題為複選,調整filename_list再loop
print(items_scoring)  #反向計分題    
print(items_m)   #複選               

['第1題', '第2題', '第3題', '第4題', '第5題', '第6題', '第7題', '第8題', '第9題', '第10題', '第11題', '第12題', '第13題', '第14題']
['第1題', '第2題', '第4題', '第5題', '第7題', '第8題', '第9題', '第10題', '第12題']
['第7題', '第9題', '第10題']
['第3題', '第6題', '第11題', '第13題', '第14題']


# def 1

In [12]:
#開檔,欄位刪除,重命名,排序,存檔輸出
import pandas as pd
import numpy as np
!pip install xlrd

def read_data(filename):
    df = pd.read_excel(filename, encoding='big5')
    df = df.rename(columns={'std_key':'學生流水碼','grad_year':'入學年','grad_diff_year':'距今畢業(年)','Q1':'第1題','Q2':'第2題','Q3':'第3題','Q4':'第4題','Q5':'第5題','Q6':'第6題','Q7':'第7題','Q8':'第8題','Q9':'第9題','Q10':'第10題','Q11':'第11題','Q12':'第12題','Q13':'第13題','Q14':'第14題','reason':'理由','remark':'備註'}) 
    df = df.drop(['理由','備註'], axis=1)
    df = df.sort_values('學生流水碼').reset_index(drop=True)
    df.to_excel('./IRdata/graduate/newdata/' + filename.split('\\')[-1].split('.xlsx')[0] + '_out.xlsx')  #創一個newdata的dir
    return df

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


# 測試

In [16]:
#代入def
for i in filename:
    read_data(i)  #檔案已輸出
    print('檔案名稱:'+ i.split('\\')[-1].split('.xlsx')[0]+'_out.xlsx')   #順便檢查次數分配,其他為NAN
    print(read_data(i).count())
    #print(read_data(i).apply(pd.value_counts).count(axis=1))   #value_count

檔案名稱:graduate_100-3y_out.xlsx
學生流水碼      1592
入學年        1592
距今畢業(年)    1592
第1題        1592
第2題        1198
第3題        1198
第4題        1198
第5題        1198
第6題         360
第7題        1198
第8題        1198
第9題        1198
第10題       1198
第11題       1198
第12題       1198
第13題        197
第14題       1592
dtype: int64
檔案名稱:graduate_101-1y_out.xlsx
學生流水碼      1352
入學年        1352
距今畢業(年)    1352
第1題        1352
第2題         842
第3題         842
第4題         842
第5題         842
第6題         480
第7題         842
第8題         842
第9題         842
第10題        842
第11題        842
第12題       1351
第13題       1352
dtype: int64
檔案名稱:graduate_101-3y_out.xlsx
學生流水碼      1490
入學年        1490
距今畢業(年)    1490
第1題        1490
第2題        1174
第3題        1174
第4題        1174
第5題        1174
第6題         314
第7題        1174
第8題        1174
第9題        1174
第10題       1174
第11題       1174
第12題       1174
第13題         12
第14題       1490
dtype: int64
檔案名稱:graduate_102-1y_out.xlsx
學生流水碼      1575
入學年        1575
距今畢業(年)  

# def  2   

In [39]:
#單選字串編碼,反向計分編碼,存檔輸出  dict

import pandas as pd
import numpy as np
!pip install xlrd

def str_recode(filename):    
    df1 = pd.read_excel(filename, encoding='big5')  #把新資料讀進來
    for i in items_s: 
        df1['r'+i] = df1[i].map(dict_2) #套用時改一下dict號, map全字符合者,不含"其他"項            
    for i in items_scoring: 
        df1['s'+i] = df1[i].map(dict_2).map({5:1,4:2,3:3,2:4,1:5})  #反向編碼,需經兩次map,前一個結果未指給新變數
    df1.to_excel('./IRdata/graduate/newdata/str_recode/' + filename.split('\\')[-1].split('.xlsx')[0] + '_str.xlsx')  
    return df1                                                                              #創一個str_recode的dir    

You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [37]:
#建立dict_1

year = [1011, 1021, 1031, 1003, 1013]
code = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
ans = [i for i in range(1,32)]
dict_1 = {}   #創一個dict存入loop結果
for i in year:
    for j in code:
        for k in ans:
            str = '%s'%i+'%s'%j+'%s'%k  #元素皆為str   
            dict = {str:k}   #dict的key-value,全字符合,否則為NAN  
            dict_1.update(dict)
dict_1

{'1011A1': 1,
 '1011A2': 2,
 '1011A3': 3,
 '1011A4': 4,
 '1011A5': 5,
 '1011A6': 6,
 '1011A7': 7,
 '1011A8': 8,
 '1011A9': 9,
 '1011A10': 10,
 '1011A11': 11,
 '1011A12': 12,
 '1011A13': 13,
 '1011A14': 14,
 '1011A15': 15,
 '1011A16': 16,
 '1011A17': 17,
 '1011A18': 18,
 '1011A19': 19,
 '1011A20': 20,
 '1011A21': 21,
 '1011A22': 22,
 '1011A23': 23,
 '1011A24': 24,
 '1011A25': 25,
 '1011A26': 26,
 '1011A27': 27,
 '1011A28': 28,
 '1011A29': 29,
 '1011A30': 30,
 '1011A31': 31,
 '1011B1': 1,
 '1011B2': 2,
 '1011B3': 3,
 '1011B4': 4,
 '1011B5': 5,
 '1011B6': 6,
 '1011B7': 7,
 '1011B8': 8,
 '1011B9': 9,
 '1011B10': 10,
 '1011B11': 11,
 '1011B12': 12,
 '1011B13': 13,
 '1011B14': 14,
 '1011B15': 15,
 '1011B16': 16,
 '1011B17': 17,
 '1011B18': 18,
 '1011B19': 19,
 '1011B20': 20,
 '1011B21': 21,
 '1011B22': 22,
 '1011B23': 23,
 '1011B24': 24,
 '1011B25': 25,
 '1011B26': 26,
 '1011B27': 27,
 '1011B28': 28,
 '1011B29': 29,
 '1011B30': 30,
 '1011B31': 31,
 '1011C1': 1,
 '1011C2': 2,
 '1011C3': 3,
 '

In [40]:
#建立dict_2, 只適用graduate_101-1y.xlsx

code = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
ans = [i for i in range(1,32)]
dict_2 = {}   
for i in code:
    for j in ans:
        str = '%s'%i +'%s'%j  
        dict = {str:j}  
        dict_2.update(dict)
dict_2

{'A1': 1,
 'A10': 10,
 'A11': 11,
 'A12': 12,
 'A13': 13,
 'A14': 14,
 'A15': 15,
 'A16': 16,
 'A17': 17,
 'A18': 18,
 'A19': 19,
 'A2': 2,
 'A20': 20,
 'A21': 21,
 'A22': 22,
 'A23': 23,
 'A24': 24,
 'A25': 25,
 'A26': 26,
 'A27': 27,
 'A28': 28,
 'A29': 29,
 'A3': 3,
 'A30': 30,
 'A31': 31,
 'A4': 4,
 'A5': 5,
 'A6': 6,
 'A7': 7,
 'A8': 8,
 'A9': 9,
 'B1': 1,
 'B10': 10,
 'B11': 11,
 'B12': 12,
 'B13': 13,
 'B14': 14,
 'B15': 15,
 'B16': 16,
 'B17': 17,
 'B18': 18,
 'B19': 19,
 'B2': 2,
 'B20': 20,
 'B21': 21,
 'B22': 22,
 'B23': 23,
 'B24': 24,
 'B25': 25,
 'B26': 26,
 'B27': 27,
 'B28': 28,
 'B29': 29,
 'B3': 3,
 'B30': 30,
 'B31': 31,
 'B4': 4,
 'B5': 5,
 'B6': 6,
 'B7': 7,
 'B8': 8,
 'B9': 9,
 'C1': 1,
 'C10': 10,
 'C11': 11,
 'C12': 12,
 'C13': 13,
 'C14': 14,
 'C15': 15,
 'C16': 16,
 'C17': 17,
 'C18': 18,
 'C19': 19,
 'C2': 2,
 'C20': 20,
 'C21': 21,
 'C22': 22,
 'C23': 23,
 'C24': 24,
 'C25': 25,
 'C26': 26,
 'C27': 27,
 'C28': 28,
 'C29': 29,
 'C3': 3,
 'C30': 30,
 'C31': 31

# 測試

In [34]:
#重開檔
!pip install glob
import glob

filename=glob.glob('./IRdata/graduate/newdata/*')   #newdata files PATH
filename 

  Could not find a version that satisfies the requirement glob (from versions: )
No matching distribution found for glob
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


['./IRdata/graduate/newdata\\graduate_100-3y_out.xlsx',
 './IRdata/graduate/newdata\\graduate_101-1y_out.xlsx',
 './IRdata/graduate/newdata\\graduate_101-3y_out.xlsx',
 './IRdata/graduate/newdata\\graduate_102-1y_out.xlsx',
 './IRdata/graduate/newdata\\graduate_103-1y_out.xlsx']

In [38]:
#代入def
for i in filename:              
    str_recode(i)  #檔案已輸出
    print('檔案名稱:'+ i.split('\\')[-1].split('.xlsx')[0]+'_str')
    #print(str_recode(i).count())  
    print(str_recode(i)[['r第1題', 'r第2題', 'r第4題', 'r第5題', 'r第7題', 'r第8題', 'r第9題', 'r第10題', 'r第12題','s第7題','s第9題','s第10題']].count()) #只印出編碼後的
    #print(str_recode(i).apply(pd.value_counts).count(axis=1))

檔案名稱:graduate_100-3y_out_str
r第1題     1582
r第2題     1198
r第4題     1191
r第5題     1183
r第7題     1198
r第8題     1198
r第9題     1198
r第10題    1198
r第12題    1190
s第7題     1198
s第9題     1198
s第10題    1198
dtype: int64
檔案名稱:graduate_101-1y_out_str
r第1題     0
r第2題     0
r第4題     0
r第5題     0
r第7題     0
r第8題     0
r第9題     0
r第10題    0
r第12題    0
s第7題     0
s第9題     0
s第10題    0
dtype: int64
檔案名稱:graduate_101-3y_out_str
r第1題     1490
r第2題     1174
r第4題     1174
r第5題     1172
r第7題     1174
r第8題     1174
r第9題     1174
r第10題    1174
r第12題    1174
s第7題     1174
s第9題     1174
s第10題    1174
dtype: int64
檔案名稱:graduate_102-1y_out_str
r第1題     1557
r第2題      760
r第4題      757
r第5題      756
r第7題      760
r第8題      760
r第9題      760
r第10題     760
r第12題     754
s第7題      760
s第9題      760
s第10題     760
dtype: int64
檔案名稱:graduate_103-1y_out_str
r第1題     2739
r第2題     1392
r第4題     1391
r第5題     1387
r第7題     1392
r第8題     1392
r第9題     1392
r第10題    1392
r第12題    1392
s第7題     1392
s第9題     1392
s第10題    1392

In [41]:
#代入def    #針對graduate_101-1y_out.xlsx

str_recode(filename[1])  #調一下dict號 
print('檔案名稱:graduate_101-1y_out_str.xlsx')
#print(str_recode(filename[1]).count()) 
print(str_recode(filename[1])[['r第1題', 'r第2題', 'r第4題', 'r第5題', 'r第7題', 'r第8題', 'r第9題', 'r第10題', 'r第12題','s第7題','s第9題','s第10題']].count()) 
#print(str_recode(filename[1]).apply(pd.value_counts).count(axis=1))

檔案名稱:graduate_101-1y_out_str.xlsx
r第1題     1320
r第2題      842
r第4題      842
r第5題      836
r第7題      842
r第8題      842
r第9題      842
r第10題     842
r第12題    1309
s第7題      842
s第9題      842
s第10題     842
dtype: int64


In [46]:
#第13題def
!pip install glob
import glob

filename=glob.glob('./IRdata/graduate/newdata/str_recode/*')  #在str_recode下面的檔案
filename  
filename=[filename[0],filename[2]]
filename

def str_recode(filename):    
    df2 = pd.read_excel(filename, encoding='big5')  #把上一個資料讀進來
    df2['r'+'第13題'] = df2['第13題'].map(dict_1)
    df2['s'+'第13題'] = df2['第13題'].map(dict_1).map({5:1,4:2,3:3,2:4,1:5})
    df2.to_excel('./IRdata/graduate/newdata/str_recode/' + filename.split('\\')[-1].split('.xlsx')[0]+'_3y.xlsx')  
    return df2

for i in filename:
    str_recode(i)  #資料已輸出原檔
    print('檔案名稱:'+ i.split('\\')[-1].split('.xlsx')[0]+'_3y.xlsx')
    #print(str_recode(i).count())
    print(str_recode(i)[['r第13題','s第13題']].count()) 
    #print(str_recode(i).apply(pd.value_counts).count(axis=1))

  Could not find a version that satisfies the requirement glob (from versions: )
No matching distribution found for glob
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


檔案名稱:graduate_100-3y_out_str_3y.xlsx
r第13題    197
s第13題    197
dtype: int64
檔案名稱:graduate_101-3y_out_str_3y.xlsx
r第13題    12
s第13題    12
dtype: int64


# def 3

In [89]:
#複選題字串編碼, 將答案存入欄位, 存檔  (以第14題為例)    

def choice(filename):     
    df3 = pd.read_excel(filename, encoding='big5')  
    for i in range(len(df3)):    #cell中整個字串loc是從左邊開始(非畫面右方),[2]空白Error  
        df3.loc[i, 'r1第14題'] = df3.loc[i, '第14題'].split('@')[0]
        df3.loc[i, 'r3第14題'] = df3.loc[i, '第14題'].split('@')[-1]   
        df3.loc[i, 'r2第14題'] = df3.loc[i, '第14題'][7:13]    
        #df3.drop_duplicates(['r1第14題', 'r2第14題'])
        #df3.drop_duplicates(['r1第14題', 'r2第14題'])    
        #再刪去重複值存成另一個dataframe                   #創一個choice dir
    df3.to_excel('./IRdata/graduate/newdata/str_recode/choice/' + filename.split('\\')[-1].split('.xlsx')[0] + '_ch.xlsx')  
    return df3                               

# 測試

In [79]:
#重開檔
filename=glob.glob('./IRdata/graduate/newdata/str_recode/*')   #str_recode下面的檔案
filename
filename=[filename[1],filename[4]]
filename

['./IRdata/graduate/newdata/str_recode\\graduate_100-3y_out_str_3y.xlsx',
 './IRdata/graduate/newdata/str_recode\\graduate_101-3y_out_str_3y.xlsx']

In [90]:
#代入def  
for i in filename: 
    choice(i).count() 
    print('檔案名稱:'+ i.split('\\')[-1].split('.xlsx')[0]+'.xlsx')
    print(choice(i)[['r1第14題','r2第14題','r3第14題']].count())   #再def刪去重複值
    #print(choice(i).apply(pd.value_counts).count(axis=1))    

檔案名稱:graduate_100-3y_out_str_3y.xlsx
r1第14題    1592
r2第14題    1592
r3第14題    1592
dtype: int64
檔案名稱:graduate_101-3y_out_str_3y.xlsx
r1第14題    1490
r2第14題    1490
r3第14題    1490
dtype: int64


# 學習成效（在校成績） 95-100

In [102]:
#查看檔案PATH
!pip install glob
import glob

filename=glob.glob('./IRdata/grades_95-100.xlsx')
filename

  Could not find a version that satisfies the requirement glob (from versions: )
No matching distribution found for glob
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


['./IRdata/grades_95-100.xlsx']

# def 4

In [103]:
#Reshape-pivot- single_value: long format轉成wide format  
import pandas as pd

def wide_format(filename):
    df = pd.read_excel(filename, encoding='big5')
    df = df.drop(['學系'], axis=1)
    df = df.sort_values(['學生流水碼','入學年','學年_學期']).reset_index(drop=True)
    df = df.pivot_table(values='平均成績',index=['入學年','學生流水碼'],columns='學年_學期')
    df.to_excel('./IRdata/' + filename.split('/')[-1].split('.xlsx')[0] + '_out.xlsx')
    return df

# test

In [105]:
#代入def
#讀取    
# print(wide_format(filename[0]).head())  #看前後幾筆,ex:入學年95者有95-1到98-2的成績
print(wide_format(filename[0]).tail())
#檢查次數分配
print(wide_format(filename[0]).count())
#print(wide_format(filename[0]).apply(pd.value_counts).count(axis=1))

學年_學期       100-1  100-2  101-1  101-2  102-1  102-2  103-1  103-2       95-1  \
入學年 學生流水碼                                                                       
95  237090    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  65.142857   
    237091    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  55.571429   
    237092    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  68.380952   
    237094    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  73.333333   
    237095    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN  74.809524   

學年_學期            95-2       96-1       96-2       97-1       97-2       98-1  \
入學年 學生流水碼                                                                      
95  237090  67.000000  69.772727  70.043478  69.500000  67.318182  77.642857   
    237091  51.952381  74.083333  57.629630  63.136364  58.818182  49.695652   
    237092  67.583333  79.296296  72.375000  69.000000  71.227273  72.800000   
    237094  70.291667  76.150000

# 學習成效（學分數）  95-100

In [106]:
#查看檔案PATH
!pip install glob
import glob

filename=glob.glob('./IRdata/credits_95-100.xlsx')
filename

  Could not find a version that satisfies the requirement glob (from versions: )
No matching distribution found for glob
You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


['./IRdata/credits_95-100.xlsx']

# def 4

In [107]:
#Reshape-pivot  -multiple_value
import pandas as pd

def wide_format(filename):
    df = pd.read_excel(filename, encoding='big5')
    df = df.drop(['學院','學系'], axis=1)
    df = df.sort_values(['學生流水碼','入學年','學年_學期']).reset_index(drop=True)
    df = df.pivot_table(values=['學期所修學分數','期中預警學分數','學期及格學分數','有預警且有及格學分數'],index=['入學年','學生流水碼'],columns='學年_學期')
    df.to_excel('./IRdata/' + filename.split('/')[-1].split('.xlsx')[0] + '_out.xlsx')
    return df

# test

In [112]:
#代入def
#讀取
print(wide_format(filename[0]).tail())  #讀取多個values
#檢查次數分配
#print(wide_format(filename[0]).count())
#print(wide_format(filename[0]).apply(pd.value_counts).count(axis=1))

           學期及格學分數                                                     ...   \
學年_學期        100-1 100-2 101-1 101-2 102-1 102-2 103-1 103-2 95-1 95-2 ...    
入學年 學生流水碼                                                              ...    
100 270079    14.0  19.0  14.0  13.0  22.0  23.0  18.0   3.0  NaN  NaN ...    
    270081    15.0  21.0  20.0  20.0  19.0  16.0  15.0  10.0  NaN  NaN ...    
    270083    20.0  21.0  26.0  22.0  24.0  21.0  16.0   9.0  NaN  NaN ...    
    270257    17.0  24.0  22.0  26.0  19.0  10.0  12.0  10.0  NaN  NaN ...    
    270549    17.0  11.0  17.0  23.0  16.0  18.0  21.0  11.0  NaN  NaN ...    

           期中預警學分數                                                      
學年_學期        100-1 100-2 101-1 101-2 102-1 102-2 103-1 103-2 99-1 99-2  
入學年 學生流水碼                                                               
100 270079     7.0  10.0   6.0  10.0   NaN   3.0   2.0   6.0  NaN  NaN  
    270081     5.0   4.0   7.0   2.0   7.0   3.0   NaN   NaN  NaN  NaN  
  